In [1]:
import pandas as pd
import numpy as np
import collections as cols
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping

2024-11-22 12:41:11.725380: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-22 12:41:11.768802: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-22 12:41:11.769524: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-22 12:41:12.528080: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#1 Cargar los datos preprocesados
file_path = 'databases/Base de datos para desarrollo v2_cat(preprocesada).csv'
df = pd.read_csv(file_path)

In [3]:
#2 Separar las características de entrada y de salida (objetivo)
input = df.drop(columns=['__categ_fin'])
output = df['__categ_fin']

print(input)
print(output)

       __temperatura   __pulso     __pas     __pad   __sat02
0           0.399555  3.522748 -2.084207 -1.004273 -2.136869
1          -0.496330  3.291283  3.483932  2.135313 -7.096196
2           0.175589  1.092363 -2.654174 -2.247026 -6.104331
3          -2.512055  1.613159 -0.286619  1.612049 -2.632802
4           0.399555  3.638481  1.028690  0.696336 -6.104331
...              ...       ...       ...       ...       ...
76805      -1.168238 -0.527894  0.897159 -0.284784 -0.153138
76806      -0.272353  0.687298  0.195661 -1.069681 -1.145004
76807      -0.272353 -0.007097 -0.461993 -0.677233  0.342795
76808       0.399555 -0.180696 -0.330462  0.303888  0.838727
76809       0.399555  0.108635 -1.470397 -0.808049  0.342795

[76810 rows x 5 columns]
0        3
1        3
2        3
3        3
4        3
        ..
76805    5
76806    5
76807    5
76808    5
76809    5
Name: __categ_fin, Length: 76810, dtype: int64


In [4]:
# Convertir la salida a categorías
output = to_categorical(output)
print(output)

[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]


In [5]:
#3 Dividir los datos en conjuntos de entrenamiento 80% y prueba 20% 
input_train, input_test, output_train, output_test = train_test_split(input, output, test_size=0.2, random_state=42)

In [6]:
#4 Construir el modelo dela red neuronal (Perceptron multicapa)
def MLP_NN():
    NumNeurons = 7
    model = Sequential()
    model.add(Dense(64, input_dim=input_train.shape[1]))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))
    model.add(Dense(32))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))
    model.add(Dense(output.shape[1], activation='softmax'))  # Usar 'softmax' para clasificación multiclase

    #opt =  keras.optimizers.Adam(learning_rate=0.001)

    # Compilar el modelo
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [7]:
#5 Entrenar el modelo
n_epochs = 500
#network = MLP_NN()
#train = network.fit(input_train, output_train, epochs=n_epochs, batch_size=32, validation_split=0.2)


network = MLP_NN()
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
train = network.fit(input_train, output_train, epochs=n_epochs, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/500


2024-11-22 12:41:13.838296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-22 12:41:13.841048: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1537/1537 [==============================] - 3s 2ms/step - loss: 1.1052 - accuracy: 0.4371 - val_loss: 1.0051 - val_accuracy: 0.4924
Epoch 2/500
1537/1537 [==============================] - 2s 1ms/step - loss: 1.0202 - accuracy: 0.4777 - val_loss: 1.0009 - val_accuracy: 0.4941
Epoch 3/500
1537/1537 [==============================] - 2s 1ms/step - loss: 1.0101 - accuracy: 0.4862 - val_loss: 0.9936 - val_accuracy: 0.5019
Epoch 4/500
1537/1537 [==============================] - 2s 1ms/step - loss: 1.0055 - accuracy: 0.4895 - val_loss: 0.9930 - val_accuracy: 0.5033
Epoch 5/500
1537/1537 [==============================] - 2s 1ms/step - loss: 1.0016 - accuracy: 0.4929 - val_loss: 0.9906 - val_accuracy: 0.5050
Epoch 6/500
1537/1537 [==============================] - 2s 1ms/step - loss: 1.0009 - accuracy: 0.4933 - val_loss: 0.9894 - val_accuracy: 0.5081
Epoch 7/500
1537/1537 [==============================] - 2s 1ms/step - loss: 0.9976 - accuracy: 0.4956 - val_loss: 0.9878 - val_accuracy: 0.51

In [8]:
#6 Evaluar el modelo
loss, accuracy = network.evaluate(input_test, output_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}")

481/481 [==============================] - 0s 933us/step - loss: 0.9820 - accuracy: 0.5145
Precisión en el conjunto de prueba: 0.51


In [9]:
#7 Predicciones
output_pred = network.predict(input_test)
print(output_pred)
output_pred_classes = np.argmax(output_pred, axis=1)
print(output_pred_classes)
output_test_classes = np.argmax(output_test, axis=1)
print(output_test_classes)


481/481 [==============================] - 0s 773us/step
[[7.8088578e-07 7.4947451e-07 7.3677489e-07 1.4743207e-01 3.3981586e-01
  5.1274985e-01]
 [3.4163497e-07 3.2701726e-07 3.1428257e-07 1.5569378e-01 3.4785512e-01
  4.9645019e-01]
 [4.5422078e-07 4.1917752e-07 5.4185676e-07 1.9074339e-01 3.7677023e-01
  4.3248495e-01]
 ...
 [6.4918152e-07 6.4173770e-07 6.6737090e-07 1.5956125e-01 3.4459558e-01
  4.9584118e-01]
 [8.6817028e-07 8.5545508e-07 8.3568000e-07 1.5719815e-01 3.4478357e-01
  4.9801567e-01]
 [1.0607201e-07 8.0907704e-08 8.2937817e-08 2.2693910e-01 4.4193897e-01
  3.3112174e-01]]
[5 5 5 ... 5 5 4]
[5 4 5 ... 4 5 3]


In [10]:
#8 Generar el reporte de clasificación
print(classification_report(output_test_classes, output_pred_classes))

              precision    recall  f1-score   support

           3       0.62      0.30      0.40      3735
           4       0.48      0.43      0.45      5771
           5       0.51      0.74      0.61      5856

    accuracy                           0.51     15362
   macro avg       0.54      0.49      0.49     15362
weighted avg       0.53      0.51      0.50     15362



In [11]:
#9 Matriz de confusión
print(confusion_matrix(output_test_classes, output_pred_classes))

[[1109 1268 1358]
 [ 583 2467 2721]
 [  86 1443 4327]]


In [12]:
# Matriz de confusión con datos de entrenamiento
output_train_pred = network.predict(input_train)
output_train_pred_classes = np.argmax(output_train_pred, axis=1)
output_train_classes = np.argmax(output_train, axis=1)
print(confusion_matrix(output_train_classes, output_train_pred_classes))

1921/1921 [==============================] - 2s 778us/step
[[ 4530  5250  5274]
 [ 2141  9711 10948]
 [  376  5905 17313]]
